In [2]:
from pykrx import stock
from pykrx import bond
import pandas as pd
import math
import datetime as dt
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU



from itertools import cycle

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import seaborn as sns

In [42]:
df = pd.read_csv('./stock_v10.csv', encoding='cp949')
df

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
0,2018-04-27,53380,53640,52440,53000,606216,1.610000e+12,1.65,1156530,16.78,...,51398.0,51648.727273,3200.0,75.110274,67.630019,49220.000000,-11.428571,3.045836e+05,65.968830,154.639175
1,2018-05-04,53000,53900,51800,51900,39565391,2.080000e+12,-2.08,28126,8.65,...,51554.0,51740.000000,1560.0,77.943844,72.360099,49308.400000,-37.735849,-1.110982e+07,59.703911,114.860864
2,2018-05-08,52600,53200,51900,52600,23104720,1.220000e+12,1.35,28126,8.77,...,51816.0,51930.181818,2620.0,74.979722,76.011280,49492.064000,-25.896414,-1.420386e+07,62.166241,103.816070
3,2018-05-09,52600,52800,50900,50900,16128305,8.310000e+11,-3.23,28126,8.49,...,51770.0,51763.636364,-460.0,57.421724,70.115097,49668.381440,-64.102564,-1.931008e+07,53.600113,48.126533
4,2018-05-10,51700,51700,50600,51600,13905263,7.120000e+11,1.38,28126,8.60,...,51652.0,51732.727273,-1180.0,53.618574,62.006673,49837.646182,-49.145299,-1.602356e+07,56.272030,31.118215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,2023-03-27,62600,62800,62000,62100,11039331,6.870000e+11,-1.43,43611,10.75,...,60900.0,61461.818182,2100.0,88.372093,85.122952,59543.206437,-27.906977,7.331683e+06,55.120378,156.416465
1207,2023-03-28,62400,62900,62100,62900,11614118,7.260000e+11,1.29,43611,10.89,...,61290.0,61825.454545,3900.0,85.271318,88.302524,59768.614050,-9.302326,9.657889e+06,58.623100,159.107271
1208,2023-03-29,62500,62700,62200,62700,11216008,7.000000e+11,-0.32,43611,10.85,...,61580.0,62081.818182,2900.0,82.945736,85.529716,59980.497207,-13.953488,1.330026e+07,57.416624,128.923476
1209,2023-03-30,63700,63700,63100,63200,15684377,9.940000e+11,0.80,43611,10.94,...,61910.0,62376.363636,3300.0,88.701963,85.639672,60179.667375,-10.638298,1.025422e+07,59.652225,158.757062


In [43]:
# SMA의 기간을 10일로 두어 MA10과 SMA가 동일함
df.drop(['업종','종목','날짜','MA10'], axis=1, inplace=True)
df

,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
0,53380,53640,52440,53000,606216,1.610000e+12,1.65,1156530,16.78,2.29,...,51398.0,51648.727273,3200.0,75.110274,67.630019,49220.000000,-11.428571,3.045836e+05,65.968830,154.639175
1,53000,53900,51800,51900,39565391,2.080000e+12,-2.08,28126,8.65,1.85,...,51554.0,51740.000000,1560.0,77.943844,72.360099,49308.400000,-37.735849,-1.110982e+07,59.703911,114.860864
2,52600,53200,51900,52600,23104720,1.220000e+12,1.35,28126,8.77,1.87,...,51816.0,51930.181818,2620.0,74.979722,76.011280,49492.064000,-25.896414,-1.420386e+07,62.166241,103.816070
3,52600,52800,50900,50900,16128305,8.310000e+11,-3.23,28126,8.49,1.81,...,51770.0,51763.636364,-460.0,57.421724,70.115097,49668.381440,-64.102564,-1.931008e+07,53.600113,48.126533
4,51700,51700,50600,51600,13905263,7.120000e+11,1.38,28126,8.60,1.83,...,51652.0,51732.727273,-1180.0,53.618574,62.006673,49837.646182,-49.145299,-1.602356e+07,56.272030,31.118215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,62600,62800,62000,62100,11039331,6.870000e+11,-1.43,43611,10.75,1.42,...,60900.0,61461.818182,2100.0,88.372093,85.122952,59543.206437,-27.906977,7.331683e+06,55.120378,156.416465
1207,62400,62900,62100,62900,11614118,7.260000e+11,1.29,43611,10.89,1.44,...,61290.0,61825.454545,3900.0,85.271318,88.302524,59768.614050,-9.302326,9.657889e+06,58.623100,159.107271
1208,62500,62700,62200,62700,11216008,7.000000e+11,-0.32,43611,10.85,1.44,...,61580.0,62081.818182,2900.0,82.945736,85.529716,59980.497207,-13.953488,1.330026e+07,57.416624,128.923476
1209,63700,63700,63100,63200,15684377,9.940000e+11,0.80,43611,10.94,1.45,...,61910.0,62376.363636,3300.0,88.701963,85.639672,60179.667375,-10.638298,1.025422e+07,59.652225,158.757062


In [44]:
from sklearn.model_selection import train_test_split

X = df
y = df['종가']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

cols = X_train.columns

In [45]:
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_estimators=100, random_state=0)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

In [46]:
importance = rf.feature_importances_
feature = X_train.columns
importances = pd.DataFrame()
importances['feature'] = feature
importances['importances'] = importance
importances.sort_values('importances', ascending=False, inplace=True)
importances.reset_index(drop=True, inplace=True)
importances

,feature,importances
0,종가,7.128818e-01
1,저가,2.067217e-01
2,고가,6.959435e-02
3,시가,7.842680e-03
4,PER,1.371676e-03
5,PBR,7.306395e-04
6,DIV,5.531951e-04
7,foreign_보유수량,6.365939e-05
8,RSI,5.621445e-05
9,MA5,4.822023e-05


In [32]:
from sklearn.feature_selection import RFE, RFECV

min_features_to_select = 10
step = 1
selector = RFECV(rf, step=step, cv=5, min_features_to_select=min_features_to_select)
selector = selector.fit(X_train, y_train)
selector.support_

array([ True,  True,  True,  True, False, False, False, False,  True,
        True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True, False, False,  True,  True, False,
       False, False, False, False, False, False])

In [33]:
a = selector.support_.reshape(1,-1)
df_a = pd.DataFrame(columns=X_train.columns, data=a)
df_a
true_columns = df_a.columns[df_a.iloc[0]].tolist()

print(true_columns)

['시가', '고가', '저가', '종가', 'PER', 'PBR', 'DIV', 'foreign_보유수량', 'MA5', 'WMA', 'MOM']


In [34]:
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVR
# X_train = np.array(X_train)
# y_train = np.array(y_train)
estimator = LinearSVR()
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)
selector.support_

C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\magne\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False,  True, False,
       False,  True, False,  True,  True,  True,  True, False, False,
       False,  True, False,  True, False, False])

In [35]:
a = selector.support_.reshape(1,-1)
df_a = pd.DataFrame(columns=X_train.columns, data=a)
df_a
true_columns = df_a.columns[df_a.iloc[0]].tolist()

print(true_columns)

['foreign_상장주식수', 'foreign_보유수량', 'foreign_한도수량', 'short_매수', 'MA5', 'MA20', 'SMA', 'WMA', 'SIG', 'ADO']


In [36]:
from sklearn.linear_model import Ridge, Lasso


ridge_alpha = 0.01
lasso_alpha = 0.001

rid = Ridge(alpha = ridge_alpha)
las = Lasso(alpha = lasso_alpha)

scaler=MinMaxScaler(feature_range=(0,1))
X_train=scaler.fit_transform(np.array(X_train))
y_train=scaler.fit_transform(np.array(y_train).reshape(-1,1))

rid.fit(X_train,y_train)
las.fit(X_train,y_train)

X_train= scaler.inverse_transform(X_train)
y_train= scaler.inverse_transform(y_train)

rid.coef_, las.coef_

C:\Users\magne\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.009669336145162043, tolerance: 0.005351701883416068
  model = cd_fast.enet_coordinate_descent(


(array([[-2.46731560e-02,  1.12947039e-01,  1.32778250e-01,
          6.30304274e-01, -5.05149657e-03,  3.32798271e-03,
          2.01317079e-02, -6.72305176e-03,  1.26795550e-03,
          2.41900825e-03,  1.52616254e-04,  1.57641691e-03,
         -8.30738562e-03,  7.86895032e-04, -1.24603661e-03,
         -2.05548837e-03,  1.57033458e-03,  5.39369700e-03,
         -1.72451425e-03,  5.15175974e-03,  2.49312045e-03,
          3.32798271e-03, -2.83226300e-03, -3.13337618e-03,
         -5.08981447e-04, -2.83226300e-03, -5.08981447e-04,
         -4.95681544e-04, -5.05149657e-03,  1.00102868e-03,
          5.97503963e-02,  3.29621695e-02,  1.51945331e-03,
          4.71129196e-02,  1.13339603e-02,  1.09279681e-02,
         -1.05178152e-02, -8.04812508e-03, -1.25976518e-03,
          4.15398946e-03,  9.47174803e-03, -1.58034328e-03]]),
 array([ 0.        ,  0.06203635,  0.28507886,  0.62112772,  0.        ,
         0.        ,  0.        ,  0.        ,  0.00192594,  0.        ,
        -0.

In [37]:
a = rid.coef_.reshape(1,-1)
df_a = pd.DataFrame(columns=cols, data=a)
df_a

,시가,고가,저가,종가,거래량,거래대금,등락률,BPS,PER,PBR,...,SMA,WMA,MOM,STCK,STCD,SIG,LWR,ADO,RSI,CCI
0,-0.024673,0.112947,0.132778,0.630304,-0.005051,0.003328,0.020132,-0.006723,0.001268,0.002419,...,0.001519,0.047113,0.011334,0.010928,-0.010518,-0.008048,-0.00126,0.004154,0.009472,-0.00158


In [38]:
aaa = df_a.columns.tolist()

fli=[]
coli=[]
for i, col in enumerate(aaa) :
    fli.append(col)
    coli.append(df_a.iloc[0,i])
    
dd = pd.DataFrame({'feature':fli, 'coef':coli})

dd.sort_values('coef', ascending=False, inplace=True)
dd.reset_index(drop=True, inplace=True)
dd

,feature,coef
0,종가,0.630304
1,저가,0.132778
2,고가,0.112947
3,MA5,0.059750
4,WMA,0.047113
5,MA20,0.032962
6,등락률,0.020132
7,MOM,0.011334
8,STCK,0.010928
9,RSI,0.009472


In [39]:

b = las.coef_.reshape(1,-1)
df_b = pd.DataFrame(columns=cols, data=b)
df_b
true_columns = df_b.columns[df_b.iloc[0] != 0].tolist()
print(true_columns)

['고가', '저가', '종가', 'PER', 'MA20', 'WMA']


RF feature_importances_ 기준 상위 10개 feature

종가, 저가, 고가, 시가, PER, PBR, DIV, foriegn_보유수량, RSI, MA5

RF RFECV 기준 11개 feature

시가, 고가, 저가, 종가, PER, PBR, DIV, foreign_보유수량, MA5, WMA, MOM

LinearSVR RFE 기준 10개 feature

foreign_상장주식수, foreign_보유수량, foreign_한도수량, short_매수, MA5, MA20, SMA, WMA, SIG, ADO

Lasso coef 기준 4개 feature

고가, 저가, 종가, PER, MA20, WMA

Ridge coef 기준 상위 10개 feature

종가 저가 고가 시가 MA5 WMA MA20 등락률 MOM STCK

------------------------------------------------------------------------

4개 포함 : 종가, 저가, 고가, MA5, WMA

3개 포함 : 시가, PER, foreign_보유수량, MA20

2개 포함 : PBR, DIV, MOM

1개 포함 : RSI, foreign_상장주식수, foreign_한도수량, short_매수, SMA, SIG, ADO, STCK, 등락률

# 종가, 저가, 고가, 시가, MA5, WMA, PER, foreign_보유수량, MA20

In [47]:
df = pd.read_csv('./stock_v11.csv', encoding='cp949')
df

,종가,WMA,MA5,PER,foreign_보유수량,RSI,MOM,MA20
0,53000,51648.72727,51580,16.78,66883763,65.968830,3200,50022.0
1,51900,51740.00000,51580,8.65,3389738000,59.703911,1560,50190.0
2,52600,51930.18182,52008,8.77,3388514370,62.166241,2620,50414.0
3,50900,51763.63636,52108,8.49,3389966037,53.600113,-460,50613.0
4,51600,51732.72727,52000,8.60,3388367976,56.272030,-1180,50756.0
...,...,...,...,...,...,...,...,...
1206,62100,61461.81818,61760,10.75,3040249034,55.120378,2100,60675.0
1207,62900,61825.45455,62280,10.89,3040001094,58.623100,3900,60795.0
1208,62700,62081.81818,62600,10.85,3040243186,57.416624,2900,60900.0
1209,63200,62376.36364,62780,10.94,3036800280,59.652225,3300,61020.0


In [49]:
df = pd.read_csv('./stock_v10.csv', encoding='cp949')
df = df.loc[:, ['종가', '저가', '고가', '시가', 'MA5', 'WMA', 'PER', 'foreign_보유수량', 'MA20']]
df

,종가,저가,고가,시가,MA5,WMA,PER,foreign_보유수량,MA20
0,53000,52440,53640,53380,51580.0,51648.727273,16.78,66883763,50022.0
1,51900,51800,53900,53000,51580.0,51740.000000,8.65,3389738000,50190.0
2,52600,51900,53200,52600,52008.0,51930.181818,8.77,3388514370,50414.0
3,50900,50900,52800,52600,52108.0,51763.636364,8.49,3389966037,50613.0
4,51600,50600,51700,51700,52000.0,51732.727273,8.60,3388367976,50756.0
...,...,...,...,...,...,...,...,...,...
1206,62100,62000,62800,62600,61760.0,61461.818182,10.75,3040249034,60675.0
1207,62900,62100,62900,62400,62280.0,61825.454545,10.89,3040001094,60795.0
1208,62700,62200,62700,62500,62600.0,62081.818182,10.85,3040243186,60900.0
1209,63200,63100,63700,63700,62780.0,62376.363636,10.94,3036800280,61020.0


In [50]:
df.to_csv('./stock_v12.csv', encoding='cp949', index=False)

In [3]:
df = pd.read_csv('./stock_v12.csv', encoding='cp949')
df

,종가,저가,고가,시가,MA5,WMA,PER,foreign_보유수량,MA20
0,53000,52440,53640,53380,51580.0,51648.727273,16.78,66883763,50022.0
1,51900,51800,53900,53000,51580.0,51740.000000,8.65,3389738000,50190.0
2,52600,51900,53200,52600,52008.0,51930.181818,8.77,3388514370,50414.0
3,50900,50900,52800,52600,52108.0,51763.636364,8.49,3389966037,50613.0
4,51600,50600,51700,51700,52000.0,51732.727273,8.60,3388367976,50756.0
...,...,...,...,...,...,...,...,...,...
1206,62100,62000,62800,62600,61760.0,61461.818182,10.75,3040249034,60675.0
1207,62900,62100,62900,62400,62280.0,61825.454545,10.89,3040001094,60795.0
1208,62700,62200,62700,62500,62600.0,62081.818182,10.85,3040243186,60900.0
1209,63200,63100,63700,63700,62780.0,62376.363636,10.94,3036800280,61020.0


# RF, KNN, LGBM 최적 변수 조합 찾기 (1,4 / 4,5 / 5,9 슬라이싱)

In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from lightgbm import LGBMRegressor

li = []
# range(4,5) / (5,9)  슬라이싱 시 변경
for r in range(1, 4):
    for combination in combinations(range(1, 9), r):
        a = list(combination)
        li.append(a)


combili = []
        
for kk in li :
    
    kk.append(0)
#     print(kk)
    df = pd.read_csv('./stock_v12.csv', encoding='cp949')

    scaler=MinMaxScaler(feature_range=(0,1))
    sdf=scaler.fit_transform(np.array(df))

    max_val = max(df['종가'])
    min_val = min(df['종가'])

    training_size=int(len(sdf)*0.70)
    test_size=len(sdf)-training_size
    train_data,test_data=sdf[0:training_size,:],sdf[training_size:len(sdf),:]

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), kk]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])

        return np.array(dataX), np.array(dataY)

    time_step = 1
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)
    X_train = X_train.reshape(X_train.shape[0],-1)
    X_test = X_test.reshape(X_test.shape[0],-1)

    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)

    train_predict=rf.predict(X_train)
    test_predict=rf.predict(X_test)

    train_predict = train_predict*(max_val-min_val) + min_val

    y_train = y_train.reshape(-1,1)
    y_train = y_train*(max_val-min_val) + min_val

    test_predict = test_predict*(max_val-min_val) + min_val

    y_test = y_test.reshape(-1,1)
    y_test = y_test*(max_val-min_val) + min_val



    rf_rmse = math.sqrt(mean_squared_error(y_test,test_predict))
    rf_r2 = r2_score(y_test,test_predict)
    

    df = pd.read_csv('./stock_v12.csv', encoding='cp949')

    scaler=MinMaxScaler(feature_range=(0,1))
    sdf=scaler.fit_transform(np.array(df))

    max_val = max(df['종가'])
    min_val = min(df['종가'])

    training_size=int(len(sdf)*0.70)
    test_size=len(sdf)-training_size
    train_data,test_data=sdf[0:training_size,:],sdf[training_size:len(sdf),:]

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), kk]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])

        return np.array(dataX), np.array(dataY)

    time_step = 1
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)
    X_train = X_train.reshape(X_train.shape[0],-1)
    X_test = X_test.reshape(X_test.shape[0],-1)


    
    neighbor = neighbors.KNeighborsRegressor(n_neighbors=15)
    neighbor.fit(X_train, y_train)

    train_predict=neighbor.predict(X_train)
    test_predict=neighbor.predict(X_test)

    train_predict = train_predict*(max_val-min_val) + min_val

    y_train = y_train.reshape(-1,1)
    y_train = y_train*(max_val-min_val) + min_val

    test_predict = test_predict*(max_val-min_val) + min_val

    y_test = y_test.reshape(-1,1)
    y_test = y_test*(max_val-min_val) + min_val


    knn_rmse = math.sqrt(mean_squared_error(y_test,test_predict))
    knn_r2 = r2_score(y_test,test_predict)

    df = pd.read_csv('./stock_v12.csv', encoding='cp949')

    scaler=MinMaxScaler(feature_range=(0,1))
    sdf=scaler.fit_transform(np.array(df))

    max_val = max(df['종가'])
    min_val = min(df['종가'])

    training_size=int(len(sdf)*0.70)
    test_size=len(sdf)-training_size
    train_data,test_data=sdf[0:training_size,:],sdf[training_size:len(sdf),:]

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), kk]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])

        return np.array(dataX), np.array(dataY)

    time_step = 1
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)
    X_train = X_train.reshape(X_train.shape[0],-1)
    X_test = X_test.reshape(X_test.shape[0],-1)


    params = {'n_estimators': 1000,
              'max_depth' : -1,
              'learning_rate': 0.1}

    lgbm = LGBMRegressor(**params).fit(X_train, y_train)

    train_predict=lgbm.predict(X_train)
    test_predict=lgbm.predict(X_test)


    train_predict = train_predict*(max_val-min_val) + min_val

    y_train = y_train.reshape(-1,1)
    y_train = y_train*(max_val-min_val) + min_val

    test_predict = test_predict*(max_val-min_val) + min_val

    y_test = y_test.reshape(-1,1)
    y_test = y_test*(max_val-min_val) + min_val



    lgbm_rmse = math.sqrt(mean_squared_error(y_test,test_predict))
    lgbm_r2 = r2_score(y_test,test_predict)
    algo_li = ['RF', 'KNN', 'LGBM']

    rmse_li = [rf_rmse, knn_rmse, lgbm_rmse]

    r2_li = [rf_r2, knn_r2, lgbm_r2]

    def listToString(str_list):
        result = ""
        for s in str_list:
            result += s + " "
        return result.strip()


    col_li = list(df.iloc[:,kk].columns)
    result = listToString(col_li)
    
    
    data = {'data' : result,
        'model':algo_li, 
            'rmse': rmse_li, 
            'r2': r2_li}

    comparison = pd.DataFrame(data)

    comparison = comparison.sort_values(['r2'], ascending=False)
    
    combili.append(comparison)
merged_df = pd.concat(combili, axis=0)
df.to_csv('./nt_fs1.csv',encoding='cp949',index=False)
#df.to_csv('./nt_fs2.csv',encoding='cp949',index=False) // 슬라이싱 시 변경

In [53]:
fs1 = pd.read_csv('./nt_fs1.csv', encoding='cp949')
fs2 = pd.read_csv('./nt_fs2.csv', encoding='cp949')
fs3 = pd.read_csv('./nt_fs3.csv', encoding='cp949')
merged_df = pd.concat([fs1,fs2,fs3], axis=0)
merged_df = merged_df.sort_values(['r2'],ascending=False)
merged_df.to_csv('./nt_fs.csv', encoding='cp949',index=False)

RF : 종가 고가 저가 PER (1130.7106   0.9728) (인덱스 0,1,2,6)
    
KNN : 종가 고가  (1009.3184   0.9783)   (인덱스 0,2)
    
LGBM : 종가 (1118.5131   0.9732)

# RF : 종가 저가 고가 PER / KNN : 종가 고가 / LGBM : 종가

# LSTM, GRU, LSTM+GRU 최적 변수 조합 찾기 (1,4 / 4,5 / 5,9 슬라이싱)

In [ ]:
from itertools import combinations
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
from tensorflow import keras

li = []

for r in range(1, 4):
    # 1부터 7까지의 수 중에서 r개를 뽑는 조합 생성
    for combination in combinations(range(1, 9), r):
        a = list(combination)
        li.append(a)


combili = []
        
for kk in li :
    
    kk.append(0)
#     print(kk)
    df = pd.read_csv('./stock_v12.csv', encoding='cp949')

    scaler=MinMaxScaler(feature_range=(0,1))
    sdf=scaler.fit_transform(np.array(df))

    max_val = max(df['종가'])
    min_val = min(df['종가'])

    training_size=int(len(sdf)*0.70)
    test_size=len(sdf)-training_size
    train_data,test_data=sdf[0:training_size,:],sdf[training_size:len(sdf),:]

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), kk]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])

        return np.array(dataX), np.array(dataY)

    time_step = 15
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)


    tf.keras.backend.clear_session()
    model=Sequential()
    model.add(GRU(32,return_sequences=True,input_shape=(15,len(kk))))
    model.add(GRU(32,return_sequences=True))
    model.add(GRU(32))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')

    stop = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1000,callbacks=[stop])


    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)
    train_predict.shape, test_predict.shape

    train_predict = train_predict*(max_val-min_val) + min_val

    y_train = y_train.reshape(-1,1)
    y_train = y_train*(max_val-min_val) + min_val

    test_predict = test_predict*(max_val-min_val) + min_val

    y_test = y_test.reshape(-1,1)
    y_test = y_test*(max_val-min_val) + min_val



    gru_rmse = math.sqrt(mean_squared_error(y_test,test_predict))
    gru_r2 = r2_score(y_test,test_predict)
    

    df = pd.read_csv('./stock_v12.csv', encoding='cp949')

    scaler=MinMaxScaler(feature_range=(0,1))
    sdf=scaler.fit_transform(np.array(df))

    max_val = max(df['종가'])
    min_val = min(df['종가'])

    training_size=int(len(sdf)*0.70)
    test_size=len(sdf)-training_size
    train_data,test_data=sdf[0:training_size,:],sdf[training_size:len(sdf),:]

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), kk]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])

        return np.array(dataX), np.array(dataY)

    time_step = 15
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    tf.keras.backend.clear_session()
    model=Sequential()
    model.add(LSTM(32,return_sequences=True,input_shape=(15,len(kk)), activation='tanh'))
    model.add(LSTM(32,return_sequences=True))
    model.add(GRU(32,return_sequences=True))
    model.add(GRU(32))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')

    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1000,callbacks=[stop])


    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)


    train_predict = train_predict*(max_val-min_val) + min_val

    y_train = y_train.reshape(-1,1)
    y_train = y_train*(max_val-min_val) + min_val

    test_predict = test_predict*(max_val-min_val) + min_val

    y_test = y_test.reshape(-1,1)
    y_test = y_test*(max_val-min_val) + min_val


    lstmgru_rmse = math.sqrt(mean_squared_error(y_test,test_predict))
    lstmgru_r2 = r2_score(y_test,test_predict)

    df = pd.read_csv('./stock_v12.csv', encoding='cp949')

    scaler=MinMaxScaler(feature_range=(0,1))
    sdf=scaler.fit_transform(np.array(df))

    max_val = max(df['종가'])
    min_val = min(df['종가'])

    training_size=int(len(sdf)*0.70)
    test_size=len(sdf)-training_size
    train_data,test_data=sdf[0:training_size,:],sdf[training_size:len(sdf),:]

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), kk]
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])

        return np.array(dataX), np.array(dataY)

    time_step = 15
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)


    tf.keras.backend.clear_session()
    model=Sequential()
    model.add(LSTM(32, return_sequences=True,input_shape=(15,len(kk)), activation='tanh'))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')

    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1000,callbacks=[stop])


    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)


    train_predict = train_predict*(max_val-min_val) + min_val

    y_train = y_train.reshape(-1,1)
    y_train = y_train*(max_val-min_val) + min_val

    test_predict = test_predict*(max_val-min_val) + min_val

    y_test = y_test.reshape(-1,1)
    y_test = y_test*(max_val-min_val) + min_val



    lstm_rmse = math.sqrt(mean_squared_error(y_test,test_predict))
    lstm_r2 = r2_score(y_test,test_predict)
    algo_li = ['lstm', 'gru', 'lstmgru']

    rmse_li = [lstm_rmse, gru_rmse, lstmgru_rmse]

    r2_li = [lstm_r2, gru_r2, lstmgru_r2]

    def listToString(str_list):
        result = ""
        for s in str_list:
            result += s + " "
        return result.strip()


    col_li = list(df.iloc[:,kk].columns)
    result = listToString(col_li)
    
    
    data = {'data' : result,
        'algorithm':['lstm', 'gru', 'lstmgru'], 
            'rmse': [lstm_rmse, gru_rmse, lstmgru_rmse], 
            'r2': [lstm_r2, gru_r2, lstmgru_r2]}

    comparison = pd.DataFrame(data)
    combili.append(comparison)

merged_df = pd.concat(combili, axis=0)
merged_df = merged_df.sort_values(['r2'], ascending=False)
merged_df.to_csv('./fs01.csv', encoding='cp949', index=False)

LSTM : 종가 고가 WMA foreign_보유수량 (908.9118 0.9827) (인덱스 0,2,5,7)
    
GRU : 종가 고가 PER foreign_보유수량 (905.4588 0.9828) (인덱스 0,2,6,7)
    
LSTM+GRU : 종가 고가 MA5 foreign_보유수량 MA20 (909.3614 0.9826) (인덱스 0,2,4,7,8)